In [4]:
from tifffile import TiffFile, imread
from pathlib import Path
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
import numpy as np
import trackpy as tp
%matplotlib widget

In [5]:
def tiff_page_generator(filename):
    """Yield each page of a TIFF file as a numpy array."""
    with TiffFile(filename) as tif:
        for page in tif.pages:
            yield page.asarray()

In [6]:
data_folder = Path("~/workspace/data/four-cell").expanduser()
pattern_file = data_folder / "20240906_patterns_Master Cy5 Cam binx2.tif"
nuclei_file = data_folder / "20240906_MCF10A_timelapse_Master DAPI Cam bin2x2.tif"
cyto_file = data_folder / "20240906_MCF10A_timelapse_Master DIA Cam bin2x2.tif"

In [7]:
pattern = imread(pattern_file)
nuclei = tiff_page_generator(nuclei_file)
cyto = tiff_page_generator(cyto_file)

In [8]:
bbox = [680, 740, 570, 630]

In [9]:
def crop(img, bbox):
    return img[bbox[0]:bbox[1], bbox[2]:bbox[3]]

In [10]:
pattern_roi = crop(pattern, bbox)

In [11]:
nuclei_roi = []
for nuclei_frame in nuclei:
    nuclei_roi.append(crop(nuclei_frame, bbox))

nuclei_roi = np.array(nuclei_roi)
print(nuclei_roi.shape)

(289, 60, 60)


In [12]:
cyto_roi = []
for cyto_frame in cyto:
    cyto_roi.append(cyto_frame[bbox[0]:bbox[1], bbox[2]:bbox[3]])

cyto_roi = np.array(cyto_roi)
print(cyto_roi.shape)

(289, 60, 60)


In [60]:
class ImageStackViewer:
    def __init__(self, img_stack, debug=False):
        self.img_stack = img_stack
        self.debug = debug
        self.slider = widgets.IntSlider(
            min=0, max=img_stack.shape[0]-1, description='Frame',
            layout=widgets.Layout(min_width='20em', border='1px solid black')
        )
        self.debug_widget = widgets.Output(
            layout=widgets.Layout(min_width='20em', min_height='15em', border='1px solid black')
        )
        self.debug_cnt = 0
        plt.ioff()
        self.fig, self.ax = plt.subplots()
        self._plot_and_track(0)
        self.ax.axis('off')
        self.fig.tight_layout()
        self.slider.observe(self._on_value_change, names='value')

        self.fig_widget = widgets.Output(
            layout=widgets.Layout(min_width='40em', min_height='40em', border='1px solid black')
        )
        with self.fig_widget:
            display(self.fig.canvas)

        self._show()

    def _on_value_change(self, change):
        frame_idx = change['new']
        self._debug(f'Frame: {frame_idx}')
        self._plot_and_track(frame_idx)
        self.fig.canvas.draw_idle()

    def _plot_and_track(self, frame_idx):
        f = tp.locate(self.img_stack[frame_idx], diameter=11, topn=10)
        self._debug(f)
        self.ax.clear()
        self.ax.imshow(self.img_stack[frame_idx], cmap='gray', vmin=self.img_stack[frame_idx].min(), vmax=self.img_stack[frame_idx].max())
        self.ax.axis('off')
        for cell in f.itertuples():
            self.ax.plot(cell.x, cell.y, 'ro', markersize=10)

    def _show(self):
        if self.debug:
            display(widgets.HBox([
                widgets.VBox([
                    self.slider,
                    widgets.Box(layout=widgets.Layout(height='1em', border='1px solid transparent')),
                    self.fig_widget
                ], layout=widgets.Layout(align_items='center')),
                self.debug_widget
            ], layout=widgets.Layout(margin='1em')))
        else:
            display(widgets.HBox([
                widgets.VBox([
                    self.slider,
                    widgets.Box(layout=widgets.Layout(height='1em', border='1px solid transparent')),
                    self.fig_widget
                ], layout=widgets.Layout(align_items='center'))
        ], layout=widgets.Layout(margin='1em')))

    def _debug(self, msg):
        self.debug_widget.clear_output()
        with self.debug_widget:
            print(msg)

In [61]:
viewer = ImageStackViewer(nuclei_roi, debug=True)